# Notebook 1: Introducción al aprendizaje por refuerzos

Primer entregable del curso Aprendizaje por Refuerzos, de la Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2023, FaMAF, UNC.

Alumno: Jerónimo Fotinós


## Actividades

1. Implementar y ejecutar el algoritmo SARSA en "The Cliff".

2. Implementar y ejecutar el algoritmo Q-Learning en "The Cliff". ¿Cómo converge con respecto a SARSA? ¿A qué se debe? Comentar.

3. Ejecutando con distintos híper-parámetros, realizar una breve descripción sobre cómo afectan a la convergencia los distintos valores de $\alpha$, $\epsilon$ y $\gamma$.

4. (Opcional) Implementar política de exploración Softmax, en donde cada acción tiene una probabilidad $$\pi(a \mid s) = \frac{e^{Q(s,a)/\tau}}{\sum_{\widetilde{a} \in A}e^{Q(s,\widetilde{a})/\tau}}$$

5. (Opcional) Implementar Dyna-Q a partir del algoritmo Q-Learning, incorporando una actualización mediante un modelo. Comentar cómo se desempeña respecto a los demás algoritmos.


Para dejar el lab listo para su corrección, dejar link a repo de Github con un notebook ejecutando el agente en la planilla enviada en Slack.

## Ambiente: The Cliff
Algunas definiciones (see [Cliff Walking - Gymnasium Documentation](https://gymnasium.farama.org/environments/toy_text/cliff_walking/))
- state: The world is a $4\times12$ grid (you should rotate your head when you see the map, so as to see it extending vertically instead of horizontally), the player starts at $[3,0]$ and the goal is at $[3,11]$, with a cliff that runs along $[3, 1 \text{ to } 10]$. There are 3 x 12 + 1 possible states. The player cannot be at the cliff, nor at the goal as the latter results in the end of the episode. The posible states are maped to a single integer (instead of a tuple) by doing `current_row * nrows + current_col` (where both the row and col start at 0). E.g. the stating position can be calculated as follows: 3 * 12 + 0 = 36.
- actions
    - 0: Move up
    - 1: Move right
    - 2: Move down
    - 3: Move left
- q: `dict` with `(state, a)` keys and `float` values (that stand for the value of taking that action in that state).
- hyperparameters: `dict` defined below, with the necessary values for SARSA and Q-learning.
- reward: -1 in every state, -100 in the cliff

![](https://github.com/JeroFotinos/rl_diplo/raw/main/images/cliffwalking.png?raw=1)

In the image, S is the starting point and G is the goal. (Imagen de Sutton y Barto, 2018.)

## Ejercicio 1

Para la resolución de este laboratorio, se implementaron los algoritmos requeridos en el script [lab_1.py](https://github.com/JeroFotinos/rl_diplo/blob/main/lab_1.py), que se encuentra en el repositorio de esta notebook. De todos modos, para facilitar la corrección copiaré las partes críticas del mismo en la notebook.

Lo primero era implementar el algoritmo SARSA.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/images/sarsa.png)

A continuación, pego el extracto de [lab_1.py](https://github.com/JeroFotinos/rl_diplo/blob/main/lab_1.py) donde se define la función. Básicamente se implementa la regla de actualización de los valores para cada acción en cada estado, pero teniendo el cuidado de inicializar los valores de $Q$ para pares estado-acción nunca antes recorridos. Dicha regla lee $$ Q^{\text{new}}(s, a) \leftarrow Q(s, a) + \alpha \times (r + \gamma \times Q(s', a') - Q(s, a)) $$donde:
- $\alpha$ es el learning rate
- $\gamma$ es el factor de descuento
- $r$ es la recompensa
- $Q(s, a)$ es el valor actual para ese par estado-acción
- $Q(s', a')$ es el valor para el siguiente par estado-acción


(Usé `black` para corregir el estilo del código y traté de seguir el estilo de NumPy para los docstrings, pero no garantizo consistencia.)

In [ ]:
def learn_SARSA(
    state: int,
    action: int,
    reward: int,
    next_state: int,
    next_action: int,
    q: dict,
    hyperparameters: dict,
) -> None:
    """Update the Q-value for the given state and action pair using SARSA
    learning (on-policy TD control).

    Parameters
    ----------
    state : int
        The current state.
    action : int
        The current action.
    reward : int
        The reward obtained by taking action `action` in state `state`.
    next_state : int
        The next state.
    next_action : int
        The next action.
    q : dict
        The Q-value table, represented as a dictionary with state-action pairs
        as keys and Q-values as values.
    hyperparameters : dict
        A dictionary of hyperparameters needed for the Q-value update.
        Expected to contain 'alpha' and 'gamma'.

    Notes
    -----
    - The function initializes the Q-value to 0.0 for new state-action pairs
    if they are not already present in `q`.
    - The Q-value for the current state-action pair is updated using the
    formula:

    .. math::
        Q(s, a) \leftarrow Q(s, a) + \alpha \times (r + \gamma \times Q(s', a') - Q(s, a))

    Where:
    - \( \alpha \) is the learning rate
    - \( \gamma \) is the discount factor
    - \( r \) is the reward
    - \( Q(s, a) \) is the current Q-value
    - \( Q(s', a') \) is the Q-value for the next state-action pair

    """

    # initialize the Q-value for brand-new state-action pairs
    if (state, action) not in q:
        q[(state, action)] = 0.0  # or another initial value
    if (next_state, next_action) not in q:
        q[(next_state, next_action)] = 0.0  # or another initial value

    # Q(s,a) <- Q(s,a) + alpha * (reward + gamma * Q(s',a') - Q(s,a))
    q[(state, action)] += hyperparameters["alpha"] * (
        reward
        + hyperparameters["gamma"] * q[(next_state, next_action)]
        - q[(state, action)]
    )

Primero se usó este algoritmo de apredizaje, junto con la estrategia $\varepsilon$-greedy con los parámetros dados, a saber: $\alpha=0.5, \gamma=1, \varepsilon=0.1$. Esto resultó en una convergencia relativamente rápida; como se puede ver en la imagen siguiente, en aproximadamente 100 episodios el agente ha aprendido la ruta segura hacia la meta.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/steps_per_episode_smooth.png)

En la imagen, la curva naranja es la versión suavizada de la curva azul que contiene los datos crudos. Decimos que converge porque los pasos por episodio se reducen hasta un valor cercano a 20, manteniéndose aproximadamente constantes luego (las variaciones se deben al ruido introducido por nuestra política estocástica).

Por otro lado, decimos que el agente aprendió la ruta segura por dos razones. Por un lado, podemos ver la matriz de acción-valor para cada estado:

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/value_matrix.png)

Siguiendo a ojo la mejor acción para cada estado, comenzando por el estado de partida, nos damos cuenta de que el agente va a tomar alguna variante del camino más seguro, i.e. el más alejado del risco. Otra manera de darnos cuenta de esto, es por inspección directa. A continuación se muestran distintos episodios del entrenamiento (utilizando el algoritmo SARSA, con estrategia $\varepsilon$-greedy con los parámetros mencionados anteriormente)

| ![Sarsa Learning, e-greedy strategy, episode 100](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/animations/episode_100.gif) | ![Sarsa Learning, e-greedy strategy, episode 200](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/animations/episode_200.gif) |
|:---:|:---:|
| Episodio 100 | Episodio 200 |
| ![Sarsa Learning, e-greedy strategy, episode 300](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/animations/episode_300.gif) | ![Sarsa Learning, e-greedy strategy, episode 400](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/sarsa_alpha=0.5_gamma=1_e=0.1/animations/episode_400.gif) |
| Episodio 300 | Episodio 400 |

**Nota 1:** para generar los GIFs, modifiqué la función `run` para que hacepte un flag booleano llamado `render`. Si se setea a `True`, se generan GIFs cada 100 episodios.

**Nota 2:** intenté que el agente aprenda a ir por el camino óptimo usando SARSA y $\varepsilon$-greedy, pero no lo conseguí. Lo más cerca que estuve de que el agente camine por el borde del risco, lo conseguí usando un $\varepsilon$ más grande, para favorecer la exploración, y un $\gamma$ mayor, para volver al agente más “largoplacista”. Esto último no trae problemas de convergencia porque los episodios son cortos dentro de todo.

## Ejercicio 2

Se implementó el algoritmo Q-Learning en “The Cliff”. 

![](https://github.com/JeroFotinos/rl_diplo/raw/main/images/q_learning.png)


A continuación, pego el extracto de [lab_1.py](https://github.com/JeroFotinos/rl_diplo/blob/main/lab_1.py) donde se define la función. Esta es análoga a la implementada para SARSA, pero ahora actualizamos el valor de una acción según el valor que el agente cree que podría haber obtenido (potencialmente, i.e. el que hubiera obtenido si a continuación tomara la mejor acción según su experiencia): $$  Q^{\text{new}}(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha [r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t)]  $$





In [ ]:
def learn_Q_learning(
    state: int,
    action: int,
    reward: int,
    next_state: int,
    next_action: int,
    q: dict,
    hyperparameters: dict,
) -> None:
    """Update the Q-value for a given state-action pair using the Q-Learning
    algorithm.

    The function performs an off-policy Temporal Difference (TD) control to
    update the Q-value for a specific state and action based on the provided
    reward and the estimated optimal future value.

    Parameters
    ----------
    state : int
        The current state in the environment, represented as an integer.
    action : int
        The action taken in the current state, represented as an integer.
    reward : int
        The immediate reward received after taking the specified action in the
        given state.
    next_state : int
        The state transitioned to after taking the specified action.
    next_action : int
        The action to be taken in the next state.
    q : dict
        A dictionary mapping from state-action tuples to Q-values.
    hyperparameters : dict
        A dictionary containing the learning rate (alpha) and discount factor
        (gamma).

    Returns
    -------
    None

    Notes
    -----
    - The `next_action` parameter is included for signature consistency with
    the SARSA function, although it is not used.
    - The function expects the action space to be of size 4, which needs to be
    considered if adapting the code for other problems.
    - The function initializes the Q-value to 0.0 for new state-action pairs.
    - The Q-value for the current state-action pair is updated using the
    formula:

    .. math::
        Q(s, a) \leftarrow Q(s, a) + \alpha \times (r + \gamma \times \max_{a'} Q(s', a') - Q(s, a))

    Where:
    - \( \alpha \) is the learning rate
    - \( \gamma \) is the discount factor
    - \( r \) is the reward
    - \( Q(s, a) \) is the current Q-value
    - \( Q(s', a') \) is the Q-value for the next state and the action that
    maximizes it

    """

    # initialize the Q-value for brand-new state-action pairs
    if (state, action) not in q:
        q[(state, action)] = 0.0  # or another initial value
    if (next_state, next_action) not in q:
        q[(next_state, next_action)] = 0.0  # or another initial value

    # Q(s,a) <- Q(s,a) + alpha * (reward + gamma * max_a' Q(s',a') - Q(s,a))
    q_max = max([q.get((next_state, a), 0.0) for a in range(4)])
    q[(state, action)] += hyperparameters["alpha"] * (
        reward + hyperparameters["gamma"] * q_max - q[(state, action)]
    )

Como puede verse en la imagen a continuación, este converge más rápidamente que SARSA. Una hipótesis de por qué es este el caso, es que en lugar de actualizar el valor de una acción (en un dado estado) según el valor que voy a extraer a futuro, lo actualizo según el máximo valor a obtener posible. Uno podría pensar que, dado que las estrategias están informadas por el valor de las acciones (y favorecen acciones de mayor valor), esto debería ser equivalente. Sin embargo, dado que las políticas son estocásticas, estas pueden ocasionalmente seleccionar una acción de menor valor. Consecuentemente, en SARSA habrán actualizaciones del valor de una acción que infravaloren la misma por haberse seleccionado una acción sub-óptima a continuación. Por el contrario, en Q-Learning actualizamos el valor de una acción en base al valor que sabemos que podríamos haber extraído. De este modo, no le cargamos el coste de la exploración a la acción tomada, pero continuamos beneficiándonos de la experiencia ganada en la exploración.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/steps_per_episode_smooth.png)

Lo que es aun más satisfactorio, es que el agente parece estar aprendiendo la ruta óptima. Nuevamente, podemos corroborar esto viendo la matriz de acción-valor para cada estado, o por inspección directa.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/value_matrix.png)

| ![Q-Learning, e-greedy strategy, episode 100](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/animations/episode_100.gif) | ![Q-Learning, e-greedy strategy, episode 200](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/animations/episode_200.gif) |
|:---:|:---:|
| Episodio 100 | Episodio 200 |
| ![Q-Learning, e-greedy strategy, episode 300](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/animations/episode_300.gif) | ![Q-Learning, e-greedy strategy, episode 400](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/q-learning_alpha=0.5_gamma=1_e=0.1_tau=25/animations/episode_400.gif) |
| Episodio 300 | Episodio 400 |


## Ejercicio 3

Ejecutando con distintos híper-parámetros, realizar una breve descripción sobre cómo afectan a la convergencia los distintos valores de $\alpha$, $\varepsilon$ y $\gamma$.

Comencemos describiendo los cambios utilizando SARSA y $\varepsilon$-greedy. En principio, esperaríamos que:
- aumentar $\alpha$ acelere la convergencia pero posiblemente aumente la varianza una vez que se logra la convergencia;
- aumentar $\varepsilon$ aumente el ruido, generando más varianza, quizás retrasando un poco la convergencia;
- aumentar $\gamma$ vuelva al agente más largoplacista, acelerando la convergencia.

Veamos si los datos corroboran o refutan estas hipótesis. Para esto, graficamos las curvas de número de pasos por episodio para los parámetros dados, cambiando de a uno. Comenzamos cambiando alpha; graficamos los entrenamientos para la siguiente lista de diccionarios.


In [ ]:
# changing alpha
hyperparameters_list = [
    {'alpha': 0.01, 'gamma': 1, 'epsilon': 0.1, 'tau': 25},
    {'alpha': 0.1, 'gamma': 1, 'epsilon': 0.1, 'tau': 25},
    {'alpha': 0.3, 'gamma': 1, 'epsilon': 0.1, 'tau': 25},
    {'alpha': 0.5, 'gamma': 1, 'epsilon': 0.1, 'tau': 25},
    ]

Con esto, obtenemos los siguientes gráficos, que representan la misma evolución en escala lineal y logarítmica respectivamente.

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/analyzing_convergence/changing_alpha_timesteps_per_episode.png)

![](https://github.com/JeroFotinos/rl_diplo/raw/main/experiments/analyzing_convergence/changing_alpha_timesteps_per_episode_log.png)

Como podemos ver, efectivamente aumentar la tasa de aprendizaje acelera la convergencia. Sin embargo, al menos aquí no se observa el aumento de varianza una vez alcanzada la misma.